# Lab 7 - Votre premier Agent Analyste de Données

Nous allons maintenant fusionner les deux mondes : la Data Science (Pandas) et l'IA Agentique (LangChain). L'objectif est de créer un agent qui peut effectuer des analyses de données à notre place.

### Étape 1 : Préparation de l'Environnement et des Données

In [ ]:
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
import os

# Assurez-vous d'avoir votre clé API OpenAI dans vos variables d'environnement
# Par exemple:
# os.environ["OPENAI_API_KEY"] = "sk-..."

df = pd.read_csv('../Lab4-DataWrangling/transactions.csv')

Un agent, comme un humain, a besoin de données de qualité pour bien travailler. Nous allons donc répéter les mêmes étapes de nettoyage que dans le Lab 4.

In [ ]:
# Supprimer les lignes où la quantité est manquante car une transaction sans quantité n'est pas exploitable
df.dropna(subset=['quantite'], inplace=True)

# Remplir le prix manquant avec la moyenne des prix du même produit (stratégie d'imputation)
df['prix_unitaire'] = df.groupby('id_produit')['prix_unitaire'].transform(lambda x: x.fillna(x.mean()))

# Convertir la colonne 'date' en datetime. errors='coerce' transformera les dates invalides en NaT (Not a Time)
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Créer une colonne 'chiffre_affaires' pour faciliter les calculs
df['chiffre_affaires'] = df['quantite'] * df['prix_unitaire']

print("Données nettoyées et prêtes pour l'agent. Aperçu :")
df.head()

### Étape 2 : Le Cerveau et le nouvel "Outil"

Voici la fonction clé : `create_pandas_dataframe_agent`. C'est un "constructeur d'agent" spécialisé. Au lieu de lui donner nos propres outils manuellement, cette fonction va analyser le DataFrame et fournir à l'agent une boîte à outils (`PythonREPLTool`) lui permettant d'écrire et d'exécuter du code Python (et donc Pandas) pour répondre à des questions.

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-4o")

# Création de l'agent
# Note: agent_type="openai-tools" est une méthode moderne et robuste qui utilise les "OpenAI Tools" pour une meilleure fiabilité.
agent_executor = create_pandas_dataframe_agent(llm, df, agent_type="openai-tools", verbose=True)

### Étape 3 : Dialoguer avec l'Agent Analyste

In [ ]:
agent_executor.invoke({"input": "Combien y a-t-il de lignes dans les données ?"})

In [ ]:
agent_executor.invoke({"input": "Quel est le chiffre d'affaires total réalisé ?"})

In [ ]:
agent_executor.invoke({"input": "Liste les catégories de produits uniques."})

In [ ]:
agent_executor.invoke({"input": "Quelle est la catégorie qui a généré le plus de chiffre d'affaires ?"})

## Conclusion

Félicitations ! Vous avez créé un véritable agent spécialisé, un "Data Analyst junior".

Le concept clé à retenir est que nous n'avons pas codé les analyses nous-mêmes. Nous avons donné au LLM un **outil** (la capacité d'exécuter du code sur un DataFrame) et un **contexte** (le DataFrame lui-même), et il a **raisonné** pour déterminer comment utiliser cet outil pour atteindre l'objectif.

Dans les prochains jours, nous apprendrons à construire des agents encore plus complexes, avec de multiples outils et des capacités de planification plus avancées.